# Fonction permettant de récupérer les films sur TMDB

## Actuellement au cinéma

### Chargement des libraires, de la clé API et délacration de la base URL

In [77]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import joblib

load_dotenv()
api_key = os.getenv("TMDB_API_KEY")

if not api_key:
    print("Erreur : clé API non trouvée.")
    exit()

base_url = "https://api.themoviedb.org/3"

df_movies = joblib.load("..\BD_A_IGNORE\df_movies.pkl")

### Récupérer les correspondances des noms de genres et ids sur TMDB

In [78]:
def get_genre_mapping(api_key):
    load_dotenv()
    api_key = os.getenv("TMDB_API_KEY")

    if not api_key:
        print("Erreur : clé API non trouvée.")
        exit()

    url = f'{base_url}/genre/movie/list?api_key={api_key}&language=en-EN'
    response = requests.get(url)
    
    if response.status_code == 200:
        genres = response.json().get("genres", [])
        return {genre["id"]: genre["name"] for genre in genres}
    else:
        print("Erreur lors de la récupération des genres.")
        return {}

In [79]:
genres_mapping = get_genre_mapping(api_key=api_key)
genres_mapping

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

### Fonction pour récupérer les films "Now Playing"(actuellement au cinéma) de TMDB

In [80]:
def get_now_playing_movies(pages=100): 
    movies = []
    for page in range(1, pages + 1):
        url = f"{base_url}/movie/now_playing?api_key={api_key}&language=fr-FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            movies.extend(data['results'])
        else:
            print(f" Erreur HTTP {response.status_code} sur la page {page}")
    return movies

In [ ]:
movies = get_now_playing_movies()
movies

### Fonction pour récupérer les détails des équipes

In [90]:
# on récupère l'id du film
def get_movie_credits(movie_id):
    url = f"{base_url}/movie/{movie_id}/credits?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# on récupère les détails du film
def get_movie_details(movie_id):
    url = f"{base_url}/movie/{movie_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# on récupère tous les équipes puis on combine tout dans une base
def fetch_movies_with_credits():
    movies = get_now_playing_movies()
    movie_data = []

    for movie in movies:
        movie_id = movie["id"]
        
        details = get_movie_details(movie_id)
        if not details:
            print(f"Impossible de récupérer les détails pour {movie_id}")
            continue

        genre_ids = movie.get("genre_ids", [])
        genres = [genres_mapping.get(genre_id) for genre_id in genre_ids if genres_mapping.get(genre_id)]

        # récupérer les crédits du film
        credits = get_movie_credits(movie_id)
        if not credits:
            print(f"Impossible de récupérer les crédits pour le film {movie_id})")
            continue

        actors = []
        actors_rank = []
        for index, cast in enumerate(credits["cast"]):
            actors.append(cast["name"])
            actors_rank.append(index + 1)

        directors = []
        for crew in credits["crew"]:
            if crew["job"] == "Director":
                directors.append(crew["name"])

        writers = []
        for crew in credits["crew"]:
            if crew["job"] == "Writer":
                writers.append(crew["name"])

        producers = []
        for crew in credits["crew"]:
            if crew["job"] == "Producer":
                producers.append(crew["name"])

        cinematographers = []
        for crew in credits["crew"]:
            if crew["job"] in ["Cinematographer", "Director of Photography"]:
                cinematographers.append(crew["name"])

        editors = []
        for crew in credits["crew"]:
            if crew["job"] == "Editor":
                editors.append(crew["name"])

        # Ajouter les données dans la liste
        movie_data.append({
            "id": movie_id,
            "originalTitle": details["original_title"],
            "release_date": details["release_date"],
            "runtimeMinutes": details.get("runtime"),
            "budget": details.get("budget"),
            "vote_average": details["vote_average"],
            "vote_count": details["vote_count"],
            "popularity": details["popularity"],
            "overview": details["overview"],
            "poster_path": details["poster_path"],
            "genres": ", ".join(genres),
            "actors": ", ".join(actors),
            "actors_rank": ", ".join(map(str, actors_rank)),
            "directors": ", ".join(directors),
            "writers": ", ".join(writers),
            "producers": ", ".join(producers),
            "cinematographers": ", ".join(cinematographers),
            "editors": ", ".join(editors),
        })

    return pd.DataFrame(movie_data)
    

In [91]:
movie_data = fetch_movies_with_credits()
movie_data

,id,originalTitle,release_date,runtimeMinutes,budget,vote_average,vote_count,popularity,overview,poster_path,genres,actors,actors_rank,directors,writers,producers,cinematographers,editors
0,696506,Mickey 17,2025-02-28,137,118000000,7.0,640,26.297,Unlikely hero Mickey Barnes finds himself in t...,/edKpE9B5qN3e559OuMCLZdW1iBZ.jpg,"Science Fiction, Comedy, Adventure","Robert Pattinson, Naomi Ackie, Steven Yeun, Ma...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",Bong Joon Ho,Bong Joon Ho,"Bong Joon Ho, Dooho Choi, Dede Gardner, Jeremy...",Darius Khondji,Yang Jin-mo
1,1286773,The Metropolitan Opera: Fidelio,2025-03-15,155,0,5.2,2,14.148,Following a string of awe-inspiring Met perfor...,/9DU8GX7tiIZQQ15MY4b44Rtgnba.jpg,Music,"Lise Davidsen, Ying Fang, David Butt Philip, M...","1, 2, 3, 4, 5, 6, 7",Jürgen Flimm,Ludwig van Beethoven,,,
2,1315476,Paradiso,2025-03-15,14,4000,8.0,1,13.625,"Inside a cold and rusty bunker, Irina and Geor...",/jN5fvKt25DtWVoPTRaEQgGmmMqd.jpg,"Drama, Horror","Lucía Zamora, Jorge Cañas, Martin Taskov, Adel...","1, 2, 3, 4",Ivan Dimitrov,"Ivan Dimitrov, Fernando Huete",Bea Guedeja-Marrón Gil,Ivan Dimitrov,Ivan Dimitrov
3,1325495,逃走,2025-03-15,110,0,0.0,0,13.606,Follows the final days of left-wing terrorist ...,/l8v8L2OC95tB4za2oex6wye4TwW.jpg,,"Kanji Furutachi, Rairu Sugita, Soran Tamoto, M...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13",Masao Adachi,Masao Adachi,Sanshirô Kobayashi,Yutaka Yamazaki,Tomoko Hiruta
4,1340727,Afterwards,2025-03-15,0,4000,0.0,0,13.587,A young man receives advice from his deceased ...,/nR5oqvfkWw94vVChRRiZRwEXhCL.jpg,"Horror, Romance","Ryan Cason, Whitney Ann Bates, Tim Knutsen, Sh...","1, 2, 3, 4, 5, 6, 7",Gibson Almgren,Gibson Almgren,Melaina Koulos,Nicholas Emmanuele,Jon Anderson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1443850,Unrecognized Character Detected,2025-02-22,9,0,0.0,0,1.035,Two alien organisms engage in a playful confli...,None,,,,,,,,
1996,1443849,Imagine the Audience Naked,2025-02-22,2,0,0.0,0,1.035,Funk band Grog Machine walk out into the spotl...,None,,,,Reuben Underwood,,,,
1997,1443103,Chucks,2025-02-22,6,0,0.0,0,1.035,A mother's wish to fulfil her son's dream of o...,None,,,,"Anum Ehtesham, Rafay Rockingson",,,,
1998,1442229,Inundation,2025-02-22,5,500,0.0,0,1.035,A poem illustrated with moving images document...,/3C8jp0ZGvHbRwjMyRj2otrEVzSu.jpg,Documentary,,,Tom Hansell,,,,


In [92]:
df_now_playing = movie_data[~movie_data["originalTitle"].isin(df_movies["originalTitle"])]

df_now_playing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1700 entries, 1 to 1999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1700 non-null   int64  
 1   originalTitle     1700 non-null   object 
 2   release_date      1700 non-null   object 
 3   runtimeMinutes    1700 non-null   int64  
 4   budget            1700 non-null   int64  
 5   vote_average      1700 non-null   float64
 6   vote_count        1700 non-null   int64  
 7   popularity        1700 non-null   float64
 8   overview          1700 non-null   object 
 9   poster_path       1451 non-null   object 
 10  genres            1700 non-null   object 
 11  actors            1700 non-null   object 
 12  actors_rank       1700 non-null   object 
 13  directors         1700 non-null   object 
 14  writers           1700 non-null   object 
 15  producers         1700 non-null   object 
 16  cinematographers  1700 non-null   object 
 17  

#### Transformer la colonne "genres" en colonnes binaires

In [93]:
def split_chaine_en_liste(x):
    if isinstance(x, str):
        return x.split(',') 
    else:
        return x

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

# transformer en vrai listes les genres
df_now_playing['genres_liste'] = df_now_playing["genres"].apply(split_chaine_en_liste)
df_now_playing['genres_liste']

# extraire les genres uniques
tous_les_genres = set()
for genres in df_now_playing["genres_liste"]:
  tous_les_genres.update(genres)

tous_les_genres

# créer des colonnes binaires pour chaque genre unique
for genre in tous_les_genres:
    def genre_present(x):
        return int(genre in x)
    
    df_now_playing[f'genre_{genre}'] = df_now_playing["genres_liste"].apply(genre_present)

df_now_playing.columns

Index(['id', 'originalTitle', 'release_date', 'runtimeMinutes', 'budget',
       'vote_average', 'vote_count', 'popularity', 'overview', 'poster_path',
       'genres', 'actors', 'actors_rank', 'directors', 'writers', 'producers',
       'cinematographers', 'editors', 'genres_liste', 'genre_',
       'genre_Western', 'genre_ Drama', 'genre_Music', 'genre_TV Movie',
       'genre_Documentary', 'genre_ Documentary', 'genre_ Horror',
       'genre_Family', 'genre_ Science Fiction', 'genre_Horror',
       'genre_ Mystery', 'genre_Science Fiction', 'genre_ Music',
       'genre_ Family', 'genre_Mystery', 'genre_History', 'genre_ Romance',
       'genre_Comedy', 'genre_ Adventure', 'genre_ Thriller', 'genre_Fantasy',
       'genre_ Action', 'genre_Crime', 'genre_Adventure', 'genre_ History',
       'genre_ Western', 'genre_Action', 'genre_Drama', 'genre_ War',
       'genre_War', 'genre_ Crime', 'genre_Animation', 'genre_ TV Movie',
       'genre_ Comedy', 'genre_ Animation', 'genre_Romance'

In [95]:
df_now_playing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1700 entries, 1 to 1999
Data columns (total 58 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1700 non-null   int64  
 1   originalTitle           1700 non-null   object 
 2   release_date            1700 non-null   object 
 3   runtimeMinutes          1700 non-null   int64  
 4   budget                  1700 non-null   int64  
 5   vote_average            1700 non-null   float64
 6   vote_count              1700 non-null   int64  
 7   popularity              1700 non-null   float64
 8   overview                1700 non-null   object 
 9   poster_path             1451 non-null   object 
 10  genres                  1700 non-null   object 
 11  actors                  1700 non-null   object 
 12  actors_rank             1700 non-null   object 
 13  directors               1700 non-null   object 
 14  writers                 1700 non-null   objec

In [96]:
# analyse des films par décennies
df_now_playing["release_date"] = pd.to_datetime(df_now_playing["release_date"], format= "%Y-%M-%d")

def decennie(date):
    date = str(date)
    if date < '1910-01-01':
        return '1910'
    elif date < '1920-01-01':
        return '1910'
    elif date < '1930-01-01':
        return '1920'
    elif date < '1940-01-01':
        return '1930'
    elif date < '1950-01-01':
        return '1940'
    elif date < '1960-01-01':
        return '1950'
    elif date < '1970-01-01':
        return '1960'
    elif date < '1980-01-01':
        return '1970'
    elif date < '1990-01-01':
        return '1980'
    elif date < '2000-01-01':
        return '1990'
    elif date < '2010-01-01':
        return '2000'
    elif date < '2020-01-01':
        return '2010'
    else: 
        return '2020'

df_now_playing["periode"] = df_now_playing["release_date"].apply(decennie)

# déterminer la position de la colonne avant laquelle je souhaite intégrer la nouvelle colonne "periode"
position_budget = df_now_playing.columns.get_loc("budget")
df_now_playing.insert(position_budget, "periode", df_now_playing.pop("periode")) # insérer la nouvelle colonne à la position souhaitée

df_now_playing["periode"] 

1       2020
3       2020
5       2020
6       2020
7       2020
        ... 
1995    2020
1996    2020
1997    2020
1998    2020
1999    2020
Name: periode, Length: 1700, dtype: object

## Films à venir "A venir"

In [97]:
def get_upcoming_movies(pages=100): 
    upcoming_movies = []
    for page in range(1, pages + 1):
        url = f"{base_url}/movie/upcoming?api_key={api_key}&language=fr-FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            upcoming_movies.extend(data['results'])
        else:
            print(f" Erreur HTTP {response.status_code} sur la page {page}")
    return upcoming_movies

In [ ]:
upcoming_movies = get_upcoming_movies()
upcoming_movies

In [99]:
def get_upcoming_movie_credits(movie_id):
    url = f"{base_url}/movie/{movie_id}/credits?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# on récupère les détails du film
def get_upcoming_movie_details(movie_id):
    url = f"{base_url}/movie/{movie_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# on récupère tous les équipes puis on combine tout dans une base
def fetch_upcoming_movies_with_credits():
    upcoming_movies = get_upcoming_movies()
    upcoming_movie_data = []

    for movie in upcoming_movies:
        movie_id = movie["id"]
        
        upcoming_details = get_upcoming_movie_details(movie_id)
        if not upcoming_details:
            print(f"Impossible de récupérer les détails pour {movie_id}")
            continue

        genre_ids = movie.get("genre_ids", [])
        genres = [genres_mapping.get(genre_id) for genre_id in genre_ids if genres_mapping.get(genre_id)]

        # récupérer les crédits du film
        upcoming_credits = get_upcoming_movie_credits(movie_id)
        if not upcoming_credits:
            print(f"Impossible de récupérer les crédits pour le film {movie_id})")
            continue

        actors = []
        actors_rank = []
        for index, cast in enumerate(upcoming_credits["cast"]):
            actors.append(cast["name"])
            actors_rank.append(index + 1)

        directors = []
        for crew in upcoming_credits["crew"]:
            if crew["job"] == "Director":
                directors.append(crew["name"])

        writers = []
        for crew in upcoming_credits["crew"]:
            if crew["job"] == "Writer":
                writers.append(crew["name"])

        producers = []
        for crew in upcoming_credits["crew"]:
            if crew["job"] == "Producer":
                producers.append(crew["name"])

        cinematographers = []
        for crew in upcoming_credits["crew"]:
            if crew["job"] in ["Cinematographer", "Director of Photography"]:
                cinematographers.append(crew["name"])

        editors = []
        for crew in upcoming_credits["crew"]:
            if crew["job"] == "Editor":
                editors.append(crew["name"])

        # Ajouter les données dans la liste
        upcoming_movie_data.append({
            "id": movie_id,
            "originalTitle": upcoming_details["original_title"],
            "release_date": upcoming_details["release_date"],
            "runtimeMinutes": upcoming_details.get("runtime"),
            "budget": upcoming_details.get("budget"),
            "vote_average": upcoming_details["vote_average"],
            "vote_count": upcoming_details["vote_count"],
            "popularity": upcoming_details["popularity"],
            "overview": upcoming_details["overview"],
            "poster_path": upcoming_details["poster_path"],
            "genres": ", ".join(genres),
            "actors": ", ".join(actors),
            "actors_rank": ", ".join(map(str, actors_rank)),
            "directors": ", ".join(directors),
            "writers": ", ".join(writers),
            "producers": ", ".join(producers),
            "cinematographers": ", ".join(cinematographers),
            "editors": ", ".join(editors),
        })

    return pd.DataFrame(upcoming_movie_data)

In [101]:
df_upcoming_movie_data = fetch_upcoming_movies_with_credits()
df_upcoming_movie_data

Impossible de récupérer les détails pour 1446794


,id,originalTitle,release_date,runtimeMinutes,budget,vote_average,vote_count,popularity,overview,poster_path,genres,actors,actors_rank,directors,writers,producers,cinematographers,editors
0,696506,Mickey 17,2025-02-28,137,118000000,7.0,640,26.297,Unlikely hero Mickey Barnes finds himself in t...,/edKpE9B5qN3e559OuMCLZdW1iBZ.jpg,"Science Fiction, Comedy, Adventure","Robert Pattinson, Naomi Ackie, Steven Yeun, Ma...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",Bong Joon Ho,Bong Joon Ho,"Bong Joon Ho, Dooho Choi, Dede Gardner, Jeremy...",Darius Khondji,Yang Jin-mo
1,1126166,Flight Risk,2025-01-22,91,25000000,6.1,493,9.331,A U.S. Marshal escorts a government witness to...,/q0bCG4NX32iIEsRFZqRtuvzNCyZ.jpg,"Action, Thriller, Crime","Mark Wahlberg, Michelle Dockery, Topher Grace,...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14",Mel Gibson,Jared Rosenberg,"John Davis, Mel Gibson, Bruce Davey",Johnny Derango,
2,1013601,The Alto Knights,2025-03-14,123,45000000,7.3,9,9.540,Two of New York's most notorious organized cri...,/95KmR0xNuZZ6DNESDwLKWGIBvMg.jpg,"Drama, Crime","Robert De Niro, Debra Messing, Cosmo Jarvis, K...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",Barry Levinson,Nicholas Pileggi,"Irwin Winkler, Jason Sosnoff, Charles Winkler,...",Dante Spinotti,Douglas Crise
3,1306707,I Don't Like It Here,2025-03-29,66,19333,0.0,0,8.209,A recently paroled outsider returns to his des...,/71tmxZLDkguFKAFOwlGtitBkkHA.jpg,"Horror, Mystery, Thriller","Jacob Nichols, Danielle Keaton, Michael Petted...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14",Robbie Smith,Robbie Smith,"Robbie Smith, Mark David, Evan Henkel",Mark David,
4,447273,Snow White,2025-03-19,109,213900000,2.0,9,12.274,"A beautiful girl, Snow White, takes refuge in ...",/xWWg47tTfparvjK0WJNX4xL8lW2.jpg,"Family, Fantasy","Rachel Zegler, Gal Gadot, Andrew Burnap, Jerem...","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",Marc Webb,,"Marc Platt, Jared LeBoff",Mandy Walker,"Mark Sanger, Sarah Broshar"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,1449583,När vi var samer,2025-03-28,180,0,0.0,0,1.400,When Mats' grandfather dies the family opens a...,/u53jNNR6qOdyJZWV9g1oMKX3p97.jpg,Drama,"Andreas Anterot, Annika Nordin, Paul Ol Jona U...","1, 2, 3, 4, 5",Johanna Sandström Lehto,Tora von Platen,,,
1168,1449513,Spicy Noodles,2025-03-29,9,0,0.0,0,1.400,Tim and Fino make a bet and eat spicy ramen. O...,/Ai2LBAs9vFUDirsXD1OnmYsZGon.jpg,,"Fin Putz, Milou Gabriel, Martina Gort","1, 2, 3",Sophia Hochedlinger,,,Leah Hochedlinger,"Leah Hochedlinger, Sophia Hochedlinger"
1169,1449319,វិញ្ញាណនិស្សិតពេទ្យ,2025-03-27,0,0,0.0,0,1.400,"In the heart of Cambodia, Mlis, a young woman ...",/bxDKQdi2NbavsiqEYRvgRHYvwQ6.jpg,Horror,"Ji Zrong, Wen Shengyuan, Keo Sinak","1, 2, 3",Kou Darachan,,,,
1170,1449316,ភ្លើងឆេះក្លឹបក្រោមដី,2025-03-20,0,0,0.0,0,1.400,Khmer horror movie.,/sM4kZw2jFncUvqbtvRFy1Bb2DPs.jpg,Horror,,,,,,,


In [102]:
def split_chaine_en_liste(x):
    if isinstance(x, str):
        return x.split(',') 
    else:
        return x
    
# transformer en vrai listes les genres
df_upcoming_movie_data['genres_liste'] = df_upcoming_movie_data["genres"].apply(split_chaine_en_liste)
df_upcoming_movie_data['genres_liste']

# extraire les genres uniques
tous_les_genres = set()
for genres in df_upcoming_movie_data["genres_liste"]:
  tous_les_genres.update(genres)

tous_les_genres

# créer des colonnes binaires pour chaque genre unique
for genre in tous_les_genres:
    def genre_present(x):
        return int(genre in x)
    
    df_upcoming_movie_data[f'genre_{genre}'] = df_upcoming_movie_data["genres_liste"].apply(genre_present)

df_upcoming_movie_data.columns

Index(['id', 'originalTitle', 'release_date', 'runtimeMinutes', 'budget',
       'vote_average', 'vote_count', 'popularity', 'overview', 'poster_path',
       'genres', 'actors', 'actors_rank', 'directors', 'writers', 'producers',
       'cinematographers', 'editors', 'genres_liste', 'genre_',
       'genre_Western', 'genre_ Drama', 'genre_Music', 'genre_Documentary',
       'genre_TV Movie', 'genre_ Documentary', 'genre_Family', 'genre_ Horror',
       'genre_ Science Fiction', 'genre_Horror', 'genre_ Mystery',
       'genre_Science Fiction', 'genre_ Music', 'genre_ Family',
       'genre_Mystery', 'genre_History', 'genre_ Romance', 'genre_Comedy',
       'genre_ Adventure', 'genre_ Thriller', 'genre_Fantasy', 'genre_ Action',
       'genre_Crime', 'genre_Adventure', 'genre_ History', 'genre_ Western',
       'genre_Action', 'genre_Drama', 'genre_ War', 'genre_War',
       'genre_Animation', 'genre_ Crime', 'genre_ TV Movie', 'genre_ Comedy',
       'genre_ Animation', 'genre_Romance'

In [103]:
df_upcoming_movie_data["release_date"] = pd.to_datetime(df_upcoming_movie_data["release_date"], format= "%Y-%M-%d")

def decennie(date):
    date = str(date)
    if date < '1910-01-01':
        return '1910'
    elif date < '1920-01-01':
        return '1910'
    elif date < '1930-01-01':
        return '1920'
    elif date < '1940-01-01':
        return '1930'
    elif date < '1950-01-01':
        return '1940'
    elif date < '1960-01-01':
        return '1950'
    elif date < '1970-01-01':
        return '1960'
    elif date < '1980-01-01':
        return '1970'
    elif date < '1990-01-01':
        return '1980'
    elif date < '2000-01-01':
        return '1990'
    elif date < '2010-01-01':
        return '2000'
    elif date < '2020-01-01':
        return '2010'
    else: 
        return '2020'

df_upcoming_movie_data["periode"] = df_upcoming_movie_data["release_date"].apply(decennie)

# déterminer la position de la colonne avant laquelle je souhaite intégrer la nouvelle colonne "periode"
ucm_position_budget = df_upcoming_movie_data.columns.get_loc("budget")
df_upcoming_movie_data.insert(ucm_position_budget, "periode", df_upcoming_movie_data.pop("periode")) # insérer la nouvelle colonne à la position souhaitée

df_upcoming_movie_data["periode"] 

0       2020
1       2020
2       2020
3       2020
4       2020
        ... 
1167    2020
1168    2020
1169    2020
1170    2020
1171    2020
Name: periode, Length: 1172, dtype: object

## Nouveau sur "NetFlix" (page d'abonnement personnel)

In [ ]:
# Index(['tconst', 'titleType', 'originalTitle', 'isAdult', 'startYear',
#        'runtimeMinutes', 'genres', 'directors', 'writers', 'averageRating',
#        'numVotes', 'periode', 'budget', 'id', 'original_language', 'overview',
#        'popularity', 'poster_path', 'production_countries', 'revenue',
#        'spoken_languages', 'vote_average', 'vote_count', 'actors_name',
#        'actors_rank', 'directors_name', 'writers_name', 'producers_name',
#        'cinematographers_name', 'editors_name', 'genres_liste',
#        'genre_History', 'genre_War', 'genre_Western', 'genre_Animation',
#        'genre_Fantasy', 'genre_Adventure', 'genre_Biography', 'genre_News',
#        'genre_Talk-Show', 'genre_Comedy', 'genre_Reality-TV', 'genre_Crime',
#        'genre_Family', 'genre_Sport', 'genre_Action', 'genre_Sci-Fi',
#        'genre_Drama', 'genre_Thriller', 'genre_Adult', 'genre_Mystery',
#        'genre_Horror', 'genre_Romance', 'genre_Documentary', 'genre_Music',
#        'genre_Film-Noir', 'genre_Unknown', 'genre_Game-Show', 'genre_Musical'],
#       dtype='object')